In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np 
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input ,Concatenate,Add
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization,Activation
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,DepthwiseConv2D,MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_h,img_w= (200,200)
batch_size=16
dirr=r"./Binarised data"


n_epochs = 500
n_cycles = n_epochs / 50

In [ ]:
def test_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), input_shape=(img_h,img_w,1)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))


  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))






  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(Dense(8, activation='softmax'))
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
def Residual_Unit(input_tensor, nb_of_input_channels, max_dilation, number_of_units):
    
  for i in range(number_of_units):
    x1 = Conv2D(nb_of_input_channels*2, 
                kernel_size = (1,1),
                strides = (1,1), padding='same', 
                dilation_rate= (1,1), activation='relu')(input_tensor)
    x1 = BatchNormalization()(x1)
  
    a = []

    for i in range(1, max_dilation+1):
      temp = DepthwiseConv2D( kernel_size=(3,3), dilation_rate = (i,i), padding = 'same', activation= 'relu')(x1)
      temp = BatchNormalization()(temp)
      a.append(temp)

    x = Concatenate(axis= -1)(a)
    x = Conv2D(nb_of_input_channels, kernel_size = (1,1), 
               strides = (1,1), padding='same', dilation_rate= (1,1), activation='relu')(x)
    x = BatchNormalization()(x)

    x = Add()([x, input_tensor])

    input_tensor = x
  
  return x

In [ ]:
def Shifter_Unit(input_tensor, nb_of_input_channels, max_dilation):
    x1 = Conv2D(nb_of_input_channels*4, kernel_size = (1,1), 
                strides = (1,1), padding='same', dilation_rate= (1,1), activation='relu')(input_tensor)
    x1 = BatchNormalization()(x1)

    a = []

    for i in range(1, max_dilation+1):
      temp = DepthwiseConv2D( kernel_size=(3,3), dilation_rate = (i,i), 
                             padding = 'same', activation= 'relu')(x1)
      temp = MaxPool2D(pool_size=(2,2))(temp)
      temp = BatchNormalization()(temp)
      a.append(temp)

    x = Concatenate(axis= -1)(a)

    x = Conv2D(nb_of_input_channels*2, kernel_size = (1,1), strides = (1,1), padding='same', dilation_rate= (1,1), activation='relu')(x)
    x = BatchNormalization()(x)

    return x

In [ ]:
def Network128(input_shape, nb_class, depth):
  xin = Input(shape= input_shape)

  x = Conv2D(16, kernel_size = (5,5), strides= (1,1), padding = 'same', activation='relu')(xin)
  x = BatchNormalization()(x)

  x = Conv2D(32, kernel_size = (3,3), strides= (2,2), padding = 'same', activation='relu')(x)
  x = BatchNormalization()(x)
  
##Max Dilation rate will be vary in the range (1,5). 

# Max Dilation rate is 5 for tensor (64x64x32)
  x = Residual_Unit(input_tensor=x, nb_of_input_channels=32, max_dilation=5, number_of_units=depth)
  x = Shifter_Unit(input_tensor=x, nb_of_input_channels=32, max_dilation=5)


# Max Dilation rate is 4 for (32x32x64)
  x = Residual_Unit(input_tensor=x, nb_of_input_channels=64, max_dilation=4, number_of_units=depth)
  x = Shifter_Unit(input_tensor=x, nb_of_input_channels=64, max_dilation=4)

# Max Dilation rate is 3 for (16x16x128)
  x = Residual_Unit(input_tensor=x, nb_of_input_channels=128, max_dilation=3, number_of_units=depth)
  x = Shifter_Unit(input_tensor=x, nb_of_input_channels=128, max_dilation=3)

# Max Dilation rate is 2 for (8x8x256)
  x = Residual_Unit(input_tensor=x, nb_of_input_channels=256, max_dilation=2, number_of_units=depth)

  x = GlobalAveragePooling2D()(x)

  x = Dense(128, activation='relu')(x)
  x = Dropout(0.3)(x)
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.3)(x)

  x = Dense(nb_class, activation= 'softmax')(x)

  model = Model(xin, x)
  opt= SGD(momentum=0.9)
  model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

  return model

In [ ]:
from tensorflow import keras
# model = Network128(input_shape = (img_h, img_w,1), nb_class = 8, depth = 2)
# /content/drive/MyDrive/thermalimaging/models1/snapshot_model_6.h5
model = keras.models.load_model('/content/drive/MyDrive/thermalimaging/models1/snapshot_model_6.h5')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 16) 416         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 200, 200, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 100, 32) 4640        batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
class SnapshotEnsemble(Callback):
    # constructor
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()

    """
    # calculate learning rate for epoch
    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = floor(n_epochs/n_cycles)
        cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (cos(cos_inner) + 1)

    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs={}):
        # calculate learning rate
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        # set learning rate
        backend.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrates.append(lr)
    """
    # save models at the end of each cycle
    def on_epoch_end(self, epoch, logs={}):
        # check if we can save model
        epochs_per_cycle = floor(self.epochs / self.cycles)
        if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
            # save model to file
            filename = "/content/drive/MyDrive/thermalimaging/snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
            self.model.save(filename)
            print('>saved snapshot %s, epoch %d' % (filename, epoch))

In [ ]:
ca = SnapshotEnsemble(n_epochs, n_cycles, 0.01)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
import numpy
from numpy import array
from numpy import argmax
 

# scale pixels
def prep_pixels(train):
	# convert from integers to floats
	train_norm = train.astype('float32')
	#test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	#test_norm = test_norm / 255.0
	# return normalized images
	return train_norm

# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = '/content/drive/MyDrive/thermalimaging/snapshot_model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models
 
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = array(yhats)
	# sum across ensemble members
	summed = numpy.sum(yhats, axis=0)
	# argmax across classes
	result = argmax(summed, axis=1)
	return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members, n_members, testX, testy):
	# select a subset of members
	subset = members[:n_members]
	# make prediction
	yhat = ensemble_predictions(subset, testX)
	# calculate accuracy
	return accuracy_score(testy, yhat)
 
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=3,
                                         cooldown=2,
                                         min_lr=1e-9,
                                         verbose=1)


In [ ]:

file_path=r'/content/drive/MyDrive/thermalimaging/IR_face_expr.csv'

data=pd.read_csv(file_path)

X= data.iloc[:,1:]  #features
y= data.iloc[:,0]
labels= list(map(int,y))

X_full= np.array(X)
y_full=np.array(y,dtype='int')

X_full = X_full.reshape((X_full.shape[0], img_h, img_w, 1))

from tensorflow.keras.utils import to_categorical
y_labels= to_categorical(y_full)


print("Shape of X is = ",X_full.shape," and Shape of labels is =",y_labels.shape)
X_train= prep_pixels(X_full)

Shape of X is =  (1782, 200, 200, 1)  and Shape of labels is = (1782, 8)


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, y_labels, test_size=0.15, random_state=42)

print("Shape of training data= ",Xtrain.shape," And labels= ",ytrain.shape)
print("Shape of Validation data= ",Xtest.shape," And labels= ",ytest.shape)

Shape of training data=  (1514, 200, 200, 1)  And labels=  (1514, 8)
Shape of Validation data=  (268, 200, 200, 1)  And labels=  (268, 8)


In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("/content/drive/MyDrive/thermalimaging"))
import cosine_annealing  as cosa

In [ ]:

#clr_triangular = cl.CyclicLR(step_size=(48*7),mode='triangular')
clr = cosa.CosineAnnealingScheduler(T_max=100, eta_max=1e-2, eta_min=1e-5)

In [ ]:
model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), batch_size=batch_size,epochs=n_epochs, verbose=1, callbacks=[clr,ca])#ca]) #reduce_learning_rate])

Epoch 1/500
95/95 [==============================] - 79s 834ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 0.9668 - val_accuracy: 0.8806
Epoch 2/500
95/95 [==============================] - 79s 828ms/step - loss: 0.0155 - accuracy: 0.9987 - val_loss: 1.0064 - val_accuracy: 0.8731
Epoch 3/500
95/95 [==============================] - 79s 829ms/step - loss: 0.0108 - accuracy: 0.9980 - val_loss: 0.9926 - val_accuracy: 0.8843
Epoch 4/500
95/95 [==============================] - 79s 830ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.9177 - val_accuracy: 0.8806
Epoch 5/500
95/95 [==============================] - 79s 828ms/step - loss: 0.0332 - accuracy: 0.9967 - val_loss: 1.1206 - val_accuracy: 0.8284
Epoch 6/500
95/95 [==============================] - 79s 827ms/step - loss: 0.0409 - accuracy: 0.9927 - val_loss: 1.0862 - val_accuracy: 0.8209
Epoch 7/500
95/95 [==============================] - 79s 829ms/step - loss: 0.0710 - accuracy: 0.9894 - val_loss: 1.0535 - val_accuracy:

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section